<a href="https://colab.research.google.com/github/greyhound101/Multihead_attention/blob/master/cascade_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive
Mounted at /content/gdrive


In [4]:
!nvidia-smi

Fri Oct 15 03:45:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!pip install medpy

     |████████████████████████████████| 151 kB 6.9 MB/s 
     |████████████████████████████████| 48.4 MB 94.4 MB/s 
  Created wheel for medpy: filename=MedPy-0.4.0-cp37-cp37m-linux_x86_64.whl size=754474 sha256=5976849975d3c08cc196eadcc9becb0cb83801031ce457c4ae48a58f96967336
  Stored in directory: /root/.cache/pip/wheels/b0/57/3a/da1183f22a6afb42e11138daa6a759de233fd977a984333602
Successfully built medpy


In [6]:
!git clone https://github.com/ZFTurbo/volumentations

Cloning into 'volumentations'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 36 (delta 8), reused 32 (delta 7), pack-reused 0
Unpacking objects: 100% (36/36), done.


In [7]:
pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 7.1 MB/s 


In [8]:
from zipfile import ZipFile
  
# specifying the zip file name
file_name = "/content/gdrive/MyDrive/results (7).zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    zip.extractall()

In [9]:
import numpy as np
t1=np.load('/content/train_0.npy')
ls=[]
for i in t1:
  ls.append('/content/'+'/'.join(i.split('/')[-2:]))

In [10]:
from zipfile import ZipFile
  
# specifying the zip file name
file_name = "/content/gdrive/MyDrive/BraTS2020_TrainingData.zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    zip.extractall()

In [11]:
file_name = "/content/gdrive/MyDrive/BraTS2020_ValidationData.zip"
with ZipFile(file_name, 'r') as zip:
    zip.extractall()

In [12]:
from medpy.io import load, save
from volumentations.volumentations import RandomCrop
from volumentations.volumentations import *
import tensorflow as tf
import glob
class DataGenerator(tf.keras.utils.Sequence):
  def __init__(self, images,batch_size=1):
    self.images = images        # array of image paths
    self.aug = Compose([
        RandomScale(p=1),
        RandomCrop((16,16,16), p=1),
        Flip(axis=0,p=0.5),
        Flip(axis=1,p=0.5),
        Flip(axis=2,p=0.5)
    ], p=1.0)
    self.on_epoch_end()
    self.batch_size=batch_size

  def __len__(self):
    return int(np.floor(len(self.images) / self.batch_size))

  def on_epoch_end(self):
    self.indexes = np.arange(len(self.images))

  def std(self,img):
    mn=np.mean(img)
    std=np.std(img)
    img=(img-mn)/std
    return img

  def augmentation(self,img,label):
    data = {'image': img, 'mask': label}
    data = self.aug(**data)
    img1,label1=data['image'],data['mask']
    return img1,label1

  def __getitem__(self, index):
    name=self.images[index].split('/')[-1]
    if self.images[index]+'/'+name+'_seg.nii'!='/content/MICCAI_BraTS2020_TrainingData/BraTS20_Training_355/BraTS20_Training_355_seg.nii':
      a=load(self.images[index]+'/'+name+'_seg.nii')[0]
    else:
      a=load('/content/MICCAI_BraTS2020_TrainingData/BraTS20_Training_355/W39_1998.09.19_Segm.nii')[0]
    path=self.images[index]+'/'+name
    b=a.copy()
    c=a.copy()
    d=a.copy()
    
    b[b!=1]=0
    c[c!=2]=0
    d[d!=4]=0
    
    c[c!=0]=1
    d[d!=0]=1
    
    img1=load(path+'_t1.nii')[0]
    img2=load(path+'_t2.nii')[0]
    img3=load(path+'_t1ce.nii')[0]
    img4=load(path+'_flair.nii')[0]
    
    img,seg=self.augmentation(self.std(np.stack([img1,img2,img3,img4],-1)),np.stack([b,c,d],-1))
    seg=np.expand_dims(seg,0)
    return np.expand_dims(img,0),[seg,seg,seg]
train=DataGenerator(ls)

In [13]:
from tensorflow.keras.layers  import *
from tensorflow_addons.layers import *
from tensorflow.keras.models import *
def block(x,l):
    x=GroupNormalization(8)(x)
    x=ReLU()(x)
    x = Conv3D(l,(3,3,3),padding='same')(x)
    return x
def create_model():
    img_input = Input(batch_shape=(None,16,16,16,4))

    x = Conv3D(16,(3,3,3),padding='same')(img_input)
    a = SpatialDropout3D(0.2)(x)

    x = block(a,16)
    x = block(x,16)

    b = Conv3D(32,(2,2,2),strides=(2,2,2))(x)

    x = block(b,32)
    x = block(x,32)


    c = Conv3D(64,(2,2,2),strides=(2,2,2))(x)

    x = block(c,64)
    x = block(x,64)

    d = Conv3D(128,(2,2,2),strides=(2,2,2))(x)

    x = block(d,128)
    x = block(x,128)

    x = Conv3D(64,(1,1,1))(x)
    x = Conv3DTranspose(64, (2,2,2), strides=(2,2,2))(x)
    x = Add()([x,c])

    x = block(x,64)
    x = block(x,64)

    x = Conv3D(32,(1,1,1))(x)
    x = Conv3DTranspose(32, (2,2,2), strides=(2,2,2))(x)
    x = Add()([x,b])

    x = block(x,32)
    x = block(x,32)

    x = Conv3D(16,(1,1,1))(x)
    x = Conv3DTranspose(16, (2,2,2), strides=(2,2,2))(x)
    x = Add()([x,a])

    x = block(x,16)
    x = block(x,16)

    x_mid = Conv3D(3,(1,1,1),activation='sigmoid')(x)

    x = Concatenate()([x_mid,img_input])

    x = Conv3D(32,(3,3,3),padding='same')(x)
    a = SpatialDropout3D(0.2)(x)

    x = block(a,32)
    x = block(x,32)

    b = Conv3D(64,(2,2,2),strides=(2,2,2))(x)

    x = block(b,64)
    x = block(x,64)


    c = Conv3D(128,(2,2,2),strides=(2,2,2))(x)

    x = block(c,128)
    x = block(x,128)

    d = Conv3D(256,(2,2,2),strides=(2,2,2))(x)

    x = block(d,256)
    enc = block(x,256)

    x = Conv3D(128,(1,1,1))(enc)
    x = Conv3DTranspose(128, (2,2,2), strides=(2,2,2))(x)
    x = Add()([x,c])

    x = block(x,128)
    x = block(x,128)

    x = Conv3D(64,(1,1,1))(x)
    x = Conv3DTranspose(64, (2,2,2), strides=(2,2,2))(x)
    x = Add()([x,b])

    x = block(x,64)
    x = block(x,64)

    x = Conv3D(32,(1,1,1))(x)
    x = Conv3DTranspose(32, (2,2,2), strides=(2,2,2))(x)
    x = Add()([x,a])

    x = block(x,32)
    x = block(x,32)

    x_ct = Conv3D(3,(1,1,1),activation='sigmoid')(x)


    x = Conv3D(128,(1,1,1))(enc)
    x = UpSampling3D(size=(2,2,2))(x)
    x = Add()([x,c])

    x = block(x,128)
    x = block(x,128)

    x = Conv3D(64,(1,1,1))(x)
    x = UpSampling3D(size=(2,2,2))(x)
    x = Add()([x,b])

    x = block(x,64)
    x = block(x,64)

    x = Conv3D(32,(1,1,1))(x)
    x = UpSampling3D(size=(2,2,2))(x)
    x = Add()([x,a])

    x = block(x,32)
    x = block(x,32)

    x_ups = Conv3D(3,(1,1,1),activation='sigmoid')(x)


    mod=Model(inputs=[img_input],outputs=[x_mid,x_ct,x_ups])
    return mod

In [14]:
from tensorflow.keras.optimizers import *
import glob
mod=create_model()
a=np.load('/content/gdrive/MyDrive/weights_e5.npy',allow_pickle=True)
mod.set_weights(a)
mod.compile(optimizer=Adam(9.99990000099999e-05),loss='bce')
mod.fit_generator(train,steps_per_epoch=1,epochs=1)
mod.optimizer.set_weights(np.load('/content/gdrive/MyDrive/opt_weights_e5.npy',allow_pickle=True))
mod.fit_generator(train,workers=8,epochs=1)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


295/295 [==============================] - 11280s 38s/step - loss: 0.0539 - conv3d_21_loss: 0.0193 - conv3d_43_loss: 0.0176 - conv3d_53_loss: 0.0170


In [15]:
np.save('/content/gdrive/MyDrive/opt_weights_e6.npy',mod.optimizer.get_weights())
np.save('/content/gdrive/MyDrive/weights_e6.npy',mod.get_weights())

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
